# 理论介绍

## MoviesLens数据集

MovieLens数据集包含多个用户对多部电影的评分数据，也包括电影元数据信息和用户属性信息。这里使用ml-1m数据,解压后有4个文件:movies.dat, users.dat, ratings.dat以及README介绍文件.

movies.dat是电影数据信息文件,表明每个电影的信息,每行数据格式为:
```
id::title::genres
# 电影编号::电影标题::电影分类
```

users.dat是用户信息文件,包含用户的ID,性别,年龄,职业,邮编地址

```
UserID::Gender::Age::Occupation::Zip-code
```

- gender: M表示男性,F表示女性
- age:
    *  1:  "Under 18" 小于18岁
    * 18:  "18-24" 
    * 25:  "25-34"
    * 35:  "35-44"
    * 45:  "45-49"
    * 50:  "50-55"
    * 56:  "56+"
- occupation: 用户职业信息
    *  0:  "other" or not specified
    *  1:  "academic/educator" 
    *  2:  "artist" 
    *  3:  "clerical/admin" 
    *  4:  "college/grad student"
    *  5:  "customer service"
    *  6:  "doctor/health care"
    *  7:  "executive/managerial"
    *  8:  "farmer"
    *  9:  "homemaker"
    * 10:  "K-12 student"
    * 11:  "lawyer"
    * 12:  "programmer"
    * 13:  "retired"
    * 14:  "sales/marketing"
    * 15:  "scientist"
    * 16:  "self-employed"
    * 17:  "technician/engineer"
    * 18:  "tradesman/craftsman"
    * 19:  "unemployed"
    * 20:  "writer"
 
ratings.dat是用户对电影评分数据文件.每行数据格式为:
```
UserID::MovieID::Rating::Timestamp
```

- UserIDs 范围在 1 - 6040
- MovieIDs 范围在 1 - 3952
- Ratings 评分数据以5星为标准 (只包含整星,不能为半星)
- Timestamp表示从1970.1.1起的秒
- 每个用户至少有20个评分数据


## UserCF

基于用户的协同过滤算法.基本思想是针对用户u来说,先根据用户历史行为数据(这里指用户的电影评分数据)找到跟用户u相似的一个用户集,然后对用户u没有评分的电影,使用用户集的行为数据,根据用户u与用户群体中用户的相似度做加权平均,得到一个**用户u没有看过电影的评分**,根据评分排序,选择topK个电影作为用户u的推荐结果.

实现UserCF主要关注几个点:

- Users-Items矩阵: 如何根据用户历史行为数据得到这个矩阵;
- 用户u的相似群体: 用户之间相似定义,评估,给出量化指标;
- 用户u的推荐电影评分: 相似用户评分的加权平均,**权重**的定义,赋值

### users-items矩阵

通过ratings.dat文件构建users-items矩阵,行表示用户u对所有电影的评分,范围是1-5;如果为0表示用户u还没有看过该电影

### 用户相似度计算

- 离散数据:![相似度计算公式](https://upload-images.jianshu.io/upload_images/9637742-8e629fe53a0da0e7.png)

- 这里采用**余弦相似度**计算公式:两个向量计算相似度

通过用户之间的相似度计算,可以得到一个用户相似度矩阵.

### 电影评分

对于用户u,
1. 根据用户相似度矩阵选择topK相似用户,形成集合S;
2. 集合S中的用户评分过的电影中筛选掉用户u看过的电影,得到候选电影集N;
3. 用户集合S和候选电影集N,推测用户u对没看过电影j的评分公式为
    - ![](https://upload-images.jianshu.io/upload_images/4155986-26d65f5981e5bbd3.png)
    - 其中,S(u,K)表示和用户u最相似的topK个用户
    - N(i):表示候选电影集
    

### 实现效果

- input: 用户u
- output: 推荐结果topN列表

## 代码实现

为了实现上述的效果,我们需要读入电影数据movies.dat以及用户评分数据ratings.dat.

- 根据ratings.dat得到users-items矩阵;
- 使用UserCF得到topN推荐结果列表,内容是ids;
- 根据topN列表电影id查询movies.dat,得到推荐电影名称

### Users-Items矩阵


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [8]:
# 读取数据
movies_path = './ml-1m/movies.dat'
ratings_path = './ml-1m/ratings.dat'

movies_cols = 'MovieID::Title::Genres'.split("::")
ratings_cols = 'UserID::MovieID::Rating::Timestamp'.split('::')

moviesDF = pd.read_csv(movies_path, sep='::', names=movies_cols)
ratingsDF = pd.read_csv(ratings_path, sep='::', names=ratings_cols)

In [12]:
# 构建users-items矩阵
# pandas pivot_table数据表透视: 行索引UserID,列索引MovieID,值Rating,填充值0
ratingPivotDF = pd.pivot_table(ratingsDF[['UserID','MovieID','Rating']],\
                                           columns=['MovieID'],index=['UserID'],\
                                           values='Rating',fill_value=0)
# 得到users字典以及movies字典,方便由id得到名称
users_map = dict(enumerate(list(ratingPivotDF.index)))
movies_map = dict(enumerate(list(ratingPivotDF.columns)))

# users-items矩阵:去除dataframe格式
ratings = ratingPivotDF.values.tolist()

### 用户相似度矩阵

In [13]:
import math
import numpy as np

# 用户之间相似度计算
def calCosineSimilarity(vec1,vec2):
    numerator = 0.0
    denominator1 = 0.0
    denominator2 = 0.0
    
    for (val1, val2) in zip(vec1, vec2):
        numerator += val1 * val2
        denominator1 += val1 ** 2
        denominator2 += val2 ** 2
    
    return numerator / math.sqrt(denominator1 * denominator2)

# 用户相似度矩阵:对称矩阵
users_similarity_matrix = np.zeros([len(ratings), len(ratings)], dtype=np.float32)
# 用户本身的相似度没有意义;同时因为对称矩阵,可以只计算上三角部分
for i in range(len(ratings)-1):#i,j:newID for users
    for j in range(i+1, len(ratings)):
        users_similarity_matrix[i,j] = calCosineSimilarity(ratings[i], ratings[j])
        users_similarity_matrix[j,i] = users_similarity_matrix[i,j]

In [14]:
# 筛选topK相似用户集合,超参数K设置为10
users_most_simi = dict()# key:userid, value: dict contailing the userID and the similarity
K = 10
for i in range(len(ratings)):# i: newID for users, or the index for users
    # 没有用到用户名称(原始用户ID)
    users_most_simi[i] = sorted(enumerate(list(ratings[i])), key=lambda x: x[1],reverse=True)[:K]

### 推荐

In [22]:
# 为每个用户推荐N个unseen电影,超参数N设置为10
N = 10
# 为unseen电影评分 unseen : ratings[i,j] = 0

user_rec_result = np.zeros([len(ratings), len(ratings[0])], dtype=np.float32)

# 遍历ratings,根据是否为0,判断seen or unseen
for i in range(len(ratings)):
    for j in range(len(ratings[0])):
        if ratings[i][j] == 0:# unseen
            val = 0
            for (u, sim) in users_most_simi[i]:
                val += sim * ratings[u][j] 
            user_rec_result[i,j] = val

# 筛选topN个推荐结果
recommend = dict()

for i in range(len(ratings)):
    recommend[i] = sorted(enumerate(list(user_rec_result[i])), key=lambda x:x[1],reverse=True)[:N]

# 查询userMap,moviesMap,找到对应的名称
recommend_list = list()

for key, value in recommend.items():
    user = users_map[key]
    for (movieID,name) in value:
        recommend_list.append([user, movies_map[name]])

recommendDF = pd.DataFrame(recommend_list,columns=['UserID','MovieID'])
recommendDF = pd.merge(recommendDF,moviesDF[['MovieID','Title']],on='MovieID',how='inner')
recommendDF.tail(10)

,UserID,MovieID,Title
60390,4486,26,Othello (1995)
60391,4486,25,Leaving Las Vegas (1995)
60392,4525,179,Mad Love (1995)
60393,4802,36,Dead Man Walking (1995)
60394,4802,36,Dead Man Walking (1995)
60395,4802,36,Dead Man Walking (1995)
60396,5100,36,Dead Man Walking (1995)
60397,5100,36,Dead Man Walking (1995)
60398,4871,193,Showgirls (1995)
60399,5910,232,Eat Drink Man Woman (1994)


In [25]:
recommendDF.head(10)

,UserID,MovieID,Title
0,1,206,Unzipped (1995)
1,875,206,Unzipped (1995)
2,960,206,Unzipped (1995)
3,1455,206,Unzipped (1995)
4,1798,206,Unzipped (1995)
5,2202,206,Unzipped (1995)
6,2284,206,Unzipped (1995)
7,2393,206,Unzipped (1995)
8,2676,206,Unzipped (1995)
9,2703,206,Unzipped (1995)
